In [1]:
import pandas as pd
import numpy as np
from numpy import array

In [2]:
def get_coefficients( File ) :
    coefficients = []
    with open(File, 'r') as model :
        for line in model :
            # performance = line.split(',')[0]
            coefficient = line.split(',')[1:]
            coefficient = [float(i) for i in coefficient]
            coefficients.append(coefficient)
    return coefficients

In [3]:
# len( coefficients )

In [4]:
model_coefficients_g = get_coefficients( './Tuning_g_Model.csv' )

In [5]:
len( model_coefficients_g )

37

In [6]:
import os 
PATH_MODEL_DATA = './MODEL_TEST_DATA/Final_Test/' 
PATH_MODEL_RANK = './MODEL_TEST_DATA/RANK/'
PATH_MODEL_RESULT = './MODEL_TEST_DATA/RESULT/'
FILE_LIST_LEN = 100

In [7]:
# type(w) : numpy.ndarray
# w.shape : 114 x 1

def score(coefficients, answer0, answer1):
    sub = answer0 - answer1
    score = sub.dot(coefficients)
    if score >= 0:
        return 1
    else :
        return -1

In [8]:
# test0
# answer_i.shape
import operator
import collections, math

def Rank_List( AnswerList, coefficients ):
    Score_Map = collections.defaultdict(lambda : 0)
    for i in range( len(AnswerList) - 1 ):

        for j in range(i+1, len(AnswerList) ):

            #print (i, j)

            answer_i = array( AnswerList.iloc[[i]].values.tolist()[0] )
            answer_j = array( AnswerList.iloc[[j]].values.tolist()[0] )

            key_i = 'A_' + str(i) + '_score'
            key_j = 'A_' + str(j) + '_score'

            value_i = 0
            value_j = 0
            value_i += score(coefficients, answer_i, answer_j) 
            value_j -= score(coefficients, answer_i, answer_j)

            Score_Map[key_i] += value_i
            Score_Map[key_j] += value_j
    
    return Score_Map
    # Sorted_scores = sorted(Score_Map.items(), key=operator.itemgetter(1), reverse=True)
    # return Sorted_scores  

In [9]:
def writeDict(Dict, filename, sep):
    with open(filename, "w") as f:
#        for key in dict.keys():            
#            f.write(str(key) + ":" + str(dict[key]) + "\n")
        for key, value in sorted( Dict.items() ):
            f.write( str(key) + sep + str(value) + '\n' )

def writeList(List, filename) :
    with open(filename, 'w') as f:
        for i in range( len(List) ) :
            # print List[i][0]
            # print List[i][1]
            f.write( str(List[i][0]) + ':' + str(List[i][1]) + '\n')
            
def readDict(filename, sep):
    with open(filename, "r") as f:
        dict = {}
        for line in f:
            values = line.split(sep)
            key = values[0]
            value = int(values[1])
            #value = values[1].split(',')
            #dict[values[0]] = {int(x) for x in values[1:len(values)]}
            dict[key] = value
        return dict 

In [10]:
# Answer_Rank = []
import operator

def update_RankList( data_path, coefficients, file_list_len = FILE_LIST_LEN, path_out = PATH_MODEL_RANK):

    for i in range(file_list_len) :
        input_file_name = 'test_' + str(i) + '.csv'
        # print input_file_name
        test_i = pd.read_csv(data_path + input_file_name , header=None)
        test_i = test_i.fillna(0)

        output_file_name_01 = 'test_' + str(i) + '_dict.txt'
        output_file_name_02 = 'test_' + str(i) + '_sorted.txt'

        # Rank test_i with given coefficients
        Ai_List = Rank_List(test_i, coefficients)
        Ai_Sorted = sorted(Ai_List.items(), key=operator.itemgetter(1), reverse=True)

        writeDict( Ai_List  , path_out  + output_file_name_01, ':')
        writeList( Ai_Sorted, path_out  + output_file_name_02 )
    # print Rank_List(test_i) 

In [11]:
# row 代表读取文件的第几行
# rank 代表要匹配这一行的答案
# 判断 sorted file 某一行的答案是否是 answer0
def hit( sorted_file , row, rank = 0, path_in = PATH_MODEL_RANK) :
    # print type(rank)
    hit = 0
    with open( path_in + sorted_file, 'r') as f:
        
        while( int(row) > 0) :
            # print row
            f.readline()
            row = row - 1
        
        answer = f.readline()
        # print answer
        answer_key = answer.split(':')[0]
        # print answer_key
        answer_rank = answer_key.split('_')[1]
        # print answer_rank
        if int(rank) == int(answer_rank) :
                hit = 1
        else :
            hit = 0
    return hit

In [12]:
def calculate_p_at_k( fout, file_list_len = FILE_LIST_LEN, path_out = PATH_MODEL_RESULT ) :    
    answer0_hit_0 = 0
    answer0_hit_1 = 0
    answer0_hit_2 = 0
    answer0_hit_3 = 0
    answer0_hit_4 = 0

    p_at_k = {}

    for i in range(file_list_len) :

        test_sorted = 'test_' + str(i) + '_sorted.txt'

        answer0_hit_0 += hit( test_sorted, 0)
        answer0_hit_1 += hit( test_sorted, 1)
        answer0_hit_2 += hit( test_sorted, 2)
        answer0_hit_3 += hit( test_sorted, 3)
        answer0_hit_4 += hit( test_sorted, 4) 
    
    p_at_k['p@1'] = answer0_hit_0
    p_at_k['p@2'] = p_at_k['p@1'] + answer0_hit_1
    p_at_k['p@3'] = p_at_k['p@2'] + answer0_hit_2
    p_at_k['p@4'] = p_at_k['p@3'] + answer0_hit_3
    p_at_k['p@5'] = p_at_k['p@4'] + answer0_hit_4
    
    writeDict( p_at_k  , path_out  + fout , ':')
    return p_at_k

In [13]:
for i in range( len(model_coefficients_g) ) :
    data_path = './MODEL_TEST_DATA/Final_Test/'
    file_list = os.listdir(data_path)
    # int file_list
    coe = model_coefficients_g[i]
    update_RankList( data_path, coe )
    fout = 'coefficients_g_' + str(i+1)
    print fout
    p_at_k = calculate_p_at_k( fout )

coefficients_g_1
coefficients_g_2
coefficients_g_3
coefficients_g_4
coefficients_g_5
coefficients_g_6
coefficients_g_7
coefficients_g_8
coefficients_g_9
coefficients_g_10
coefficients_g_11
coefficients_g_12
coefficients_g_13
coefficients_g_14
coefficients_g_15
coefficients_g_16
coefficients_g_17
coefficients_g_18
coefficients_g_19
coefficients_g_20
coefficients_g_21
coefficients_g_22
coefficients_g_23
coefficients_g_24
coefficients_g_25
coefficients_g_26
coefficients_g_27
coefficients_g_28
coefficients_g_29
coefficients_g_30
coefficients_g_31
coefficients_g_32
coefficients_g_33
coefficients_g_34
coefficients_g_35
coefficients_g_36
coefficients_g_37


In [14]:
model_coefficients_u = get_coefficients( './Tuning_u_Model.csv' )

In [15]:
len( model_coefficients_u )

19

In [16]:
for i in range( len(model_coefficients_u) ) :
    data_path = './MODEL_TEST_DATA/Final_Test/'
    file_list = os.listdir(data_path)
    # int file_list
    coe = model_coefficients_g[i]
    update_RankList( data_path, coe )
    fout = 'coefficients_u_' + str(i+1)
    print fout
    p_at_k = calculate_p_at_k( fout )

coefficients_u_1
coefficients_u_2
coefficients_u_3
coefficients_u_4
coefficients_u_5
coefficients_u_6
coefficients_u_7
coefficients_u_8
coefficients_u_9
coefficients_u_10
coefficients_u_11
coefficients_u_12
coefficients_u_13
coefficients_u_14
coefficients_u_15
coefficients_u_16
coefficients_u_17
coefficients_u_18
coefficients_u_19
